## Energy Community Forecasting
### Final Project 
#### SUPSI DTI 25/26 | Time Series, Analytics and Forecasting | D3A
#### De Appolonia Lorenzo - Fernandez Diz Sergio - Testoni Luca

### INTRODUCTION

For the final challenge you’re request to form **groups of 2-3 persons** and hand in a time series analysis and forecasting pipeline.

1. Chose a dataset among the proposed one
2. For the chosen dataset, forecast the target through the following steps:
    1. EXPLORATORY ANALYSIS. E.g. data plotting, correlation plots, ACF, etc..
    2. MODEL BUILDING. E.g. check stationarity, transform, choose model & fit, check residuals
    3. CV and model SELECTION. 
    4. Presentation of results

Keep your code:
- **Clean and well-organized**
- **Efficient** – avoid extremely long runtimes; (if it takes 1 hour to run assume it is not usable, alternatively, warn the user and explain very well why it should take so long)
- **Commented** – your code should be easy to understand without guessing.
- **Short -** Chatgpt implementations tend to be long, I would like to correct **your** code ****

### Deadline
- *You can hand in a notebook or (extensively) commented **running** code, by **Jan 19 2026***

### Hard filter
- **Your best model performing worse than naive persistent is sufficient condition for failing the exam**

Energy communities are a way of indirectly increase the self consumption and flatten the variance of the power profile of a group of consumers/producers. The members of the community receive economic incentives if their self consumption is maximized and their joint profile is flattened. If deferrable devices, like electric boilers, or electric storage, can be operated, this maximization requires to forecast both single profiles and their aggregation.

The dataset contains real data from 22 measurement points concerning an energy community in Ticino in the AEM’s grid. The 15 minutes sampling time series come from the following sources:
- 20 residential houses
- a district-level battery, which was operational during the data acquisition campaign
- the point of common coupling (PCC) with the main grid. This is close to, but not equal, the sum of the other measurements. The reason of the discrepancy being mainly due to power losses.

Each measurement point has both a `e_pos`  and `e_neg` signals, which refers to the energy withdrawn or injected back to the grid. Since the measurements are averages over 15 minutes,  `e_pos`  and `e_neg` are not mutually exclusive, meaning they can be both non-zero in the same timestamp.
Additional measurements that are provided with the datasets are:
- Weather measurements from a local weather station including solar irradiance and temperature
- NWP forecasts provided each 12 hours for different steps ahead

### Challenge:

The task is to produce 1-24 hours ahead forecasts at 1h resolution of  `e_pos`  + `e_neg` for **all the residential buildings and the PCC.**

**Provide a probabilistic forecast for the PCC**


## IMPORTS

In [ ]:
## IMPORTS 
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import math
import lightgbm as lgb
import warnings
from chronos import ChronosPipeline


warnings.filterwarnings(
    "ignore",
    message="X does not have valid feature names"
)

## 1. Import dataset

In [ ]:
## IMPORTS DATA 

df_meters = pd.read_pickle("lic_meters")
df_meteo = pd.read_pickle("lic_meteo")
df_nwp = pd.read_pickle("lic_nwp")

## SHAPES
print(
    f"Meters  : {df_meters.shape}\n"
    f"Meteo   : {df_meteo.shape}\n"
    f"NWP     : {df_nwp.shape}"
)

## CLEAR DISALLIGNMENT OF THE SERIES POSSIBLY CAUSED BOTH SAMPLING FREQUENCY AND TIME DOMAIN

## 2. Exploration Analysis

### 2.1. Data Visualization

In [ ]:
df_meters.head(2)

In [ ]:
df_meteo.head(2)

In [ ]:
df_nwp.head(2)

### 2.2. Check Time Allignment

In [ ]:
def time_summary(name, df):
    idx = df.index

    return {
        "dataset": name,
        "start": idx.min(),
        "end": idx.max(),
        "rows": len(idx),
        "freq (median)": idx.to_series().diff().median(),
        "freq (mode)": idx.to_series().diff().mode().iloc[0],
    }

summary = pd.DataFrame([
    time_summary("Meters", df_meters),
    time_summary("Meteo", df_meteo),
    ## time_summary("NWP", df_nwp),
]).set_index("dataset")

summary

In [ ]:
## alligne the all data with our target dataset ( meters )
t_start = df_meters.index.min()
t_end = df_meters.index.max()

df_meters = df_meters.loc[t_start:t_end]
df_meteo = df_meteo.loc[t_start:t_end]
## df_nwp = df_nwp.loc[t_start:t_end]

summary_2 = pd.DataFrame([
    time_summary("Meters", df_meters),
    time_summary("Meteo", df_meteo),
    ## time_summary("NWP", df_nwp),
]).set_index("dataset")

summary_2

### 2.3. Missing Values

In [ ]:
print(f'\nTot missing value in meters data: {df_meters.isnull().sum().sum()}\n')
print(df_meters.info())
df_meters.describe()

In [ ]:
print(f'\nTot missing value in meteo data: {df_meteo.isnull().sum().sum()}\n')
print(df_meteo.info())
df_meteo.describe()

In [ ]:
print(f'\nTot missing value - meteo data: {df_nwp.isnull().sum().sum()}')
print(df_nwp.info())
df_nwp.describe()

### 2.4. Missing Values - Interpolation

RESAMPLE TO HOURLY THE DATAFRAMES AND INTEROPOLATE MISSING VALUES

In [ ]:
## SUM FOR ALL METERS e_pos AND e_neg in df_energy
df_energy = df_meters.xs("e_pos", level=1, axis=1) + df_meters.xs("e_neg", level=1, axis=1)

## RESAMPLE TO HOURLY 
df_energy = (df_energy.resample("1h").sum()) ## using sum for energy over time period 
df_meteo = (df_meteo.resample("1h").mean())  ## using mean for physical mesurments over time period 

## INTERPOLATE THE FEW MISSING VALUES
df_energy = (df_energy.interpolate(method="time"))
df_meteo = (df_meteo.interpolate(method="time"))

In [ ]:
summary_3 = pd.DataFrame([
    time_summary("Meters", df_energy),
    time_summary("Meteo", df_meteo),
    ## time_summary("NWP", df_nwp),
]).set_index("dataset")

summary_3

meters data and meteo data are now perfectly alligned for modelling purposes

### 2.5. Data Plotting 

#### 2.5.1. Energy Meters 

In [ ]:
meters_to_plot = [0, 1, 2, 3, 4, "battery", "PCC"]

fig = go.Figure()
for meter in meters_to_plot:
    fig.add_trace(
        go.Scatter(
            x=df_meters.index,
            y=df_meters[(meter, "e_pos")],
            mode="lines",
            name=f"{meter} e_pos",
        )
    )

fig.update_layout(
    title="Energy Time Series (Selected Meters – e_pos)",
    xaxis_title="Time",
    yaxis_title="Energy (e_pos)",
    template="plotly_white",
    legend_title="Meter",
)
fig.show()


In [ ]:
meters_to_plot = [0, 1, 2, 3, 4, "battery", "PCC"]

fig = go.Figure()
for meter in meters_to_plot:
    fig.add_trace(
        go.Scatter(
            x=df_meters.index,
            y=df_meters[(meter, "e_neg")],
            mode="lines",
            name=f"{meter} e_neg",
        )
    )

fig.update_layout(
    title="Energy Time Series (Selected Meters – e_neg)",
    xaxis_title="Time",
    yaxis_title="Energy (e_neg)",
    template="plotly_white",
    legend_title="Meter",
)
fig.show()


#### 2.5.2. Total Energy

In [ ]:
meters_to_plot = [0, 1, 2, 3, 4, "battery", "PCC"]

fig = go.Figure()
for meter in meters_to_plot:
    fig.add_trace(
        go.Scatter(
            x=df_energy.index,
            y=df_energy[meter],
            mode="lines",
            name=f"{meter}",
        )
    )

fig.update_layout(
    title="Total Energy Time Series (Selected Meters – engineered)",
    xaxis_title="Time",
    yaxis_title="Total energy",
    template="plotly_white",
    legend_title="Meter",
)
fig.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display

def plot_energy_distribution(meter):
    s = df_energy[meter].dropna()

    mean = s.mean()
    std  = s.std()
    min_v = s.min()
    max_v = s.max()

    fig = go.Figure()
    fig.add_trace(go.Histogram(x=s, nbinsx=60, opacity=0.8, showlegend=False))

    fig.update_layout(
        title=f"Total Energy Distribution — {meter}",
        template="plotly_white",
        height=520,
        bargap=0.05,
        xaxis_title="Total energy (df_energy)",
        yaxis_title="Count",
        xaxis=dict(range=[min_v, max_v]),
    )

    fig.add_vline(x=mean, line_color="red", line_dash="dash",
                  annotation_text="Mean", annotation_position="top")
    fig.add_vline(x=mean + std, line_color="orange", line_dash="dot",
                  annotation_text="+1 std", annotation_position="top")
    fig.add_vline(x=mean - std, line_color="orange", line_dash="dot",
                  annotation_text="-1 std", annotation_position="top")
    fig.add_vline(x=min_v, line_color="blue", line_dash="dash",
                  annotation_text="Min", annotation_position="top")
    fig.add_vline(x=max_v, line_color="blue", line_dash="dash",
                  annotation_text="Max", annotation_position="top")

    fig.show()


In [ ]:
meter_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=19,
    step=1,
    description="House:",
    continuous_update=False
)

special_meter = widgets.ToggleButtons(
    options=["house", "battery", "PCC"],
    description="Type:",
)

def update_plot(house, Type):
    if Type == "house":
        plot_energy_distribution(house)
    else:
        plot_energy_distribution(Type)

ui = widgets.VBox([special_meter, meter_slider])
out = widgets.interactive_output(update_plot, {
    "house": meter_slider,
    "Type": special_meter
})

display(ui, out)


#### 2.5.3 Meteo Measurements

In [ ]:
meteo_to_plot = ['AirPressure', 'AirTemp_Avg', 'PyrIrradiance_Avg', 'Ramount_Tot', 'RelHumidity', 'WindSpeed']

fig = go.Figure()
for meteo in meteo_to_plot:
    fig.add_trace(
        go.Scatter(
            x=df_meteo.index,
            y=df_meteo[meteo],
            mode="lines",
            name=str(meteo),
        )
    )

fig.update_layout(
    title="Meteo Time Series (raw sensors)",
    xaxis_title="Time",
    yaxis_title="Value",
    template="plotly_white",
    legend_title="Sensor",
)
fig.show()


In [ ]:
## plot distributions 
features = meteo_to_plot
n_features = len(features)

# Grid size
n_cols = 3
n_rows = int(np.ceil(n_features / n_cols))

fig = make_subplots(rows=n_rows, cols=n_cols, subplot_titles=features)

for i, feat in enumerate(features):
    row = i // n_cols + 1
    col = i % n_cols + 1

    data = df_meteo[feat].dropna()

    mean = data.mean()
    std = data.std()
    min_v = data.min()
    max_v = data.max()

    # Histogram
    fig.add_trace(
        go.Histogram(
            x=data,
            nbinsx=50,
            name=feat,
            showlegend=False,
            opacity=0.75
        ),
        row=row,
        col=col
    )

    # Mean
    fig.add_vline(x=mean, line_color="red", line_dash="dash", row=row, col=col)

    # ± std
    fig.add_vline(x=mean + std, line_color="orange", line_dash="dot", row=row, col=col)
    fig.add_vline(x=mean - std, line_color="orange", line_dash="dot", row=row, col=col)

    # Min / Max
    fig.add_vline(x=min_v, line_color="blue", line_dash="dash", row=row, col=col)
    fig.add_vline(x=max_v, line_color="blue", line_dash="dash", row=row, col=col)
    fig.update_xaxes(
        range=[min_v, max_v],
        row=row,
        col=col
    )

fig.update_layout(
    title="Meteo Feature Distributions with Statistics",
    template="plotly_white",
    height=300 * n_rows,
    bargap=0.05
)

fig.show()

### 2.6. Autocorrelation 

#### 2.6.1. Energy Dataset

In [ ]:
def autocorr(x, max_lag):
    x = x[~np.isnan(x)]
    if len(x) == 0:
        return np.zeros(max_lag + 1)
    x = x - np.mean(x)
    r = np.correlate(x, x, mode="full")
    acf = r[r.size // 2:]
    if acf[0] == 0:
        return np.zeros(max_lag + 1)
    return acf[:max_lag + 1] / acf[0]

def plot_acf_meter(meter, max_lag=168, df=df_energy):
    s = df[meter].values
    acf_vals = autocorr(s, max_lag)
    lags = np.arange(len(acf_vals))

    fig = go.Figure(go.Bar(x=lags, y=acf_vals))
    fig.update_layout(
        title=f"ACF — {meter} (max_lag={max_lag})",
        template="plotly_white",
        height=450,
        xaxis_title="Lag",
        yaxis_title="ACF",
        showlegend=False
    )
    fig.show()

In [ ]:
house = widgets.IntSlider(0, 0, 19, 1, description="House:", continuous_update=False)
kind  = widgets.ToggleButtons(options=["house", "battery", "PCC"], description="Type:")
lag   = widgets.IntSlider(168, 24, 336, 1, description="max_lag:", continuous_update=False)

def update_plot(House, Type, max_lag):
    meter = House if Type == "house" else Type
    plot_acf_meter(meter=meter, max_lag=max_lag, df=df_energy)

display(widgets.VBox([kind, house, lag]),
        widgets.interactive_output(update_plot, {"House": house, "Type": kind, "max_lag": lag}))

#### 2.6.2. Meteo Dataset

In [ ]:
def plot_acf_meteo(feature, max_lag=168):
    s = df_meteo[feature].values
    acf_vals = autocorr(s, max_lag)
    lags = np.arange(len(acf_vals))

    fig = go.Figure(go.Bar(x=lags, y=acf_vals, marker_color="orange"))
    fig.update_layout(
        title=f"ACF — {feature}",
        template="plotly_white",
        height=450,
        xaxis_title="Lag",
        yaxis_title="ACF",
        showlegend=False
    )
    fig.show()

In [ ]:
feature_slider = widgets.SelectionSlider(
    options=df_meteo.columns.tolist(),
    value=df_meteo.columns[0],
    description="Meteo:",
    continuous_update=False
)

lag_slider = widgets.IntSlider(
    value=168,
    min=24,
    max=336,
    step=1,
    description="max_lag:",
    continuous_update=False
)

display(
    widgets.VBox([feature_slider, lag_slider]),
    widgets.interactive_output(
        plot_acf_meteo,
        {"feature": feature_slider, "max_lag": lag_slider}
    )
)


### 2.7. Correlation between ENRGY and METEO 

In [ ]:
## check meteo and energy correlation
pcc_series = df_energy["PCC"] 
corr_df = pd.concat([pcc_series.rename("PCC"), df_meteo], axis=1, join="inner")

print("Index equal:", corr_df.index.equals(pcc_series.index.intersection(df_meteo.index)))
print("Frequency:", pd.infer_freq(corr_df.index))
print("Missing values:", corr_df.isna().sum().sum())

corr_matrix = corr_df.corr()

fig, ax = plt.subplots(figsize=(10, 6))
im = ax.imshow(corr_matrix.values,cmap="coolwarm",vmin=-1,vmax=1)
cbar = plt.colorbar(im, ax=ax)
cbar.set_label("Correlation")
ax.set_xticks(np.arange(len(corr_matrix.columns)))
ax.set_yticks(np.arange(len(corr_matrix.index)))
ax.set_xticklabels(corr_matrix.columns, rotation=45, ha="right")
ax.set_yticklabels(corr_matrix.index)

# Annotate each cell with correlation value
for i in range(corr_matrix.shape[0]):
    for j in range(corr_matrix.shape[1]):
        value = corr_matrix.iloc[i, j]
        ax.text(j, i,f"{value:.2f}",ha="center",va="center",color="black" if abs(value) < 0.6 else "white",fontsize=9)
        
ax.set_title("Correlation Matrix: PCC vs Meteo")
plt.tight_layout()
plt.show()

## 3. Models

### 3.1. Benchmark model - Seasonal Persistence

In [ ]:
meters = [m for m in df_energy.columns if m != "battery"]

split_idx = int(len(df_energy) * 0.7)

train = df_energy.iloc[:split_idx]
test  = df_energy.iloc[split_idx:]

In [ ]:
def daily_persistence_forecast(series, horizon=24):
    return series.shift(24).iloc[-horizon:].values

def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def evaluate_meter_persistence(series_train, series_test, horizon=24):
    errors = []

    full_series = pd.concat([series_train, series_test])

    test_start = len(series_train)

    for t in range(test_start, len(full_series) - horizon):
        history = full_series.iloc[:t]
        y_true = full_series.iloc[t : t + horizon].values
        y_pred = history.shift(24).iloc[-horizon:].values

        if np.any(np.isnan(y_pred)):
            continue

        errors.append(mae(y_true, y_pred))

    return np.mean(errors)

In [ ]:
meter_errors = {}

for meter in meters:
    err = evaluate_meter_persistence(
        train[meter],
        test[meter],
        horizon=24
    )
    meter_errors[meter] = err
    print(f'PERSISTENT MAE meter {meter}: {round(err,3)}')
global_error = np.mean(list(meter_errors.values()))
print(f'\nAvareged PERSISTENT MAE across all meters: {round(global_error, 3)} ')

In [ ]:
def plot_persistence_window(meter, t, horizon=24):
    series = pd.concat([train[meter], test[meter]])

    history = series.iloc[t-horizon:t]
    y_true  = series.iloc[t:t+horizon]
    y_pred  = history.values

    if len(history) < horizon:
        return

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=history.index, y=history.values,
        mode="lines+markers",
        name="History (model input)",
        line=dict(color="black")
    ))

    fig.add_trace(go.Scatter(
        x=y_true.index, y=y_pred,
        mode="lines+markers",
        name="Prediction",
        line=dict(color="red", dash="dash")
    ))

    fig.add_trace(go.Scatter(
        x=y_true.index, y=y_true.values,
        mode="lines+markers",
        name="True",
        line=dict(color="blue")
    ))

    fig.update_layout(
        title=f"Daily Persistence Forecast — Meter {meter}",
        template="plotly_white",
        xaxis_title="Time",
        yaxis_title="Energy",
        height=500,
        auto_size=True,
    )
    fig.show(config={"responsive": True})
    

In [ ]:
meter = "PCC"  
horizon = 24

series_len = len(train[meter]) + len(test[meter])

t_slider = widgets.IntSlider(
    value=len(train[meter]),
    min=horizon,
    max=series_len - horizon,
    step=1,
    description="t:",
    continuous_update=False
)

play = widgets.Play(
    value=t_slider.value,
    min=t_slider.min,
    max=t_slider.max,
    step=t_slider.step,
    interval=300
)

widgets.jslink((play, "value"), (t_slider, "value"))

ui = widgets.HBox([play, t_slider])

out = widgets.interactive_output(
    plot_persistence_window,
    {"meter": widgets.fixed(meter), "t": t_slider, "horizon": widgets.fixed(horizon)}
)

display(ui, out)


### 3.2. L-GBM Model

#### 3.2.1. Univariate

In [ ]:
def build_lagged_dataset(series, lags=24, horizon=24):
    X, Y = [], []

    values = series.values

    for t in range(lags, len(values) - horizon):
        X.append(values[t - lags : t])
        Y.append(values[t : t + horizon])

    return np.array(X), np.array(Y)

def train_test_data(series_train, series_test, lags=24, horizon=24):
    X_train, y_train = build_lagged_dataset(series_train, lags, horizon)

    combined = pd.concat([series_train.tail(lags), series_test])
    X_test, y_test = build_lagged_dataset(combined, lags, horizon)
    
    X_train = np.asarray(X_train)
    X_test = np.asarray(X_test)
    y_train = np.asarray(y_train)
    y_test = np.asarray(y_test)

    return X_train, y_train, X_test, y_test

def train_lgbm_models(X_train, y_train):
    models = []

    for h in range(y_train.shape[1]):
        model = lgb.LGBMRegressor(
            n_estimators=150,
            learning_rate=0.05,
            random_state=42,
            verbose=-1, 
            force_col_wise=True 
        )
        model.fit(X_train, y_train[:, h])
        models.append(model)

    return models

def evaluate_lgbm(models, X_test, y_test):
    preds = np.column_stack([
        model.predict(X_test) for model in models
    ])

    return mae(y_test, preds)

In [ ]:
lgbm_errors = {}
all_lgbm_models = {}

for meter in meters:
    X_train, y_train, X_test, y_test = train_test_data(train[meter], test[meter],24,24)

    models = train_lgbm_models(X_train, y_train)
    all_lgbm_models[meter] = models
    err = evaluate_lgbm(models, X_test, y_test)

    lgbm_errors[meter] = err
    print(f"LGBM MAE meter {meter}: {round(err, 3)}")

print("\nAverage LGBM MAE across all meters:", round(np.mean(list(lgbm_errors.values())),3))

In [ ]:
def plot_lgbm_window(meter, t, models_dict, lags=24, horizon=24):
    series = pd.concat([train[meter], test[meter]])

    history = series.iloc[t-lags:t]
    y_true  = series.iloc[t:t+horizon]

    if len(history) < lags or len(y_true) < horizon:
        return

    X = history.values.reshape(1, -1)
    models = models_dict[meter]
    y_pred = np.array([models[h].predict(X)[0] for h in range(horizon)])

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=history.index, y=history.values,
        mode="lines+markers",
        name="History (model input)",
        line=dict(color="black")
    ))

    fig.add_trace(go.Scatter(
        x=y_true.index, y=y_pred,
        mode="lines+markers",
        name="LGBM Prediction",
        line=dict(color="red", dash="dash")
    ))

    fig.add_trace(go.Scatter(
        x=y_true.index, y=y_true.values,
        mode="lines+markers",
        name="True",
        line=dict(color="blue")
    ))

    fig.update_layout(
        title=f"LGBM Forecast — Meter {meter}",
        template="plotly_white",
        xaxis_title="Time",
        yaxis_title="Energy",
        height=500,
        autosize = True
    )
    fig.show(config={"responsive": True})

In [ ]:
meter = "PCC"
lags = 24
horizon = 24

series_len = len(train[meter]) + len(test[meter])

t_slider = widgets.IntSlider(
    value=len(train[meter]),
    min=lags,
    max=series_len - horizon,
    step=1,
    description="t:",
    continuous_update=False
)

play = widgets.Play(
    value=t_slider.value,
    min=t_slider.min,
    max=t_slider.max,
    step=t_slider.step,
    interval=300
)

widgets.jslink((play, "value"), (t_slider, "value"))

ui = widgets.HBox([play, t_slider])

out = widgets.interactive_output(
    plot_lgbm_window,
    {
        "meter": widgets.fixed(meter),
        "t": t_slider,
        "models_dict": widgets.fixed(all_lgbm_models),
        "lags": widgets.fixed(lags),
        "horizon": widgets.fixed(horizon),
    }
)

display(ui, out)

#### 3.2.2. Multivariate

In [ ]:
def build_calendar_features(index):
    cal = pd.DataFrame(index=index)
    cal["hour"] = index.hour
    cal["dayofweek"] = index.dayofweek
    cal["is_weekend"] = (index.dayofweek >= 5).astype(int)
    cal["month"] = index.month
    return cal

In [ ]:
def build_lagged_dataset_multivariate(target_series, meteo_df, lags=24, horizon=24, meteo_features=None):
    if meteo_features is None:
        meteo_features = []

    X, Y = [], []

    cal_df = build_calendar_features(target_series.index)

    for t in range(lags, len(target_series) - horizon):

        y_lags = target_series.iloc[t - lags:t].values
        cal_feats = cal_df.iloc[t].values
        meteo_feats = meteo_df.iloc[t][meteo_features].values

        X.append(np.concatenate([y_lags, cal_feats, meteo_feats]))
        Y.append(target_series.iloc[t:t + horizon].values)

    return np.array(X), np.array(Y)

In [ ]:
def train_test_data_multivariate(series_train, series_test, meteo_train, meteo_test, lags=24, horizon=24, meteo_features=None):
    
    X_train, y_train = build_lagged_dataset_multivariate(series_train, meteo_train, lags, horizon, meteo_features)
    combined_series = pd.concat([series_train.tail(lags), series_test])
    combined_meteo = pd.concat([meteo_train.tail(lags), meteo_test])
    X_test, y_test = build_lagged_dataset_multivariate(combined_series, combined_meteo, lags, horizon, meteo_features)

    return X_train, y_train, X_test, y_test

In [ ]:
# Define features
meteo_features = ["AirTemp_Avg", "PyrIrradiance_Avg"]

# Dictionary to store models for each meter
lgbm_multivariate_models = {}
lgbm_multivariate_errors = {}

# Loop over each meter to train a dedicated model
for meter in meters:
    
    X_train, y_train, X_test, y_test = train_test_data_multivariate(train[meter], test[meter], df_meteo.loc[train.index], df_meteo.loc[test.index], lags=24, horizon=24,  meteo_features=meteo_features)
    
    # Train the models
    models = train_lgbm_models(X_train, y_train)
    lgbm_multivariate_models[meter] = models
    
    # Evaluate 
    err = evaluate_lgbm(models, X_test, y_test)
    lgbm_multivariate_errors[meter] = err
    print(f"LGBM multivariate MAE meter {meter}: {round(err, 3)}")

print(f"\nAverage LGBM Multivariate MAE across all meters: {np.mean(list(lgbm_multivariate_errors.values())):.3f}")

In [ ]:
def plot_lgbm_multi_window(meter, t, models_dict, meteo_df, meteo_features, lags=24, horizon=24):
    series = pd.concat([train[meter], test[meter]])
    meteo  = meteo_df.reindex(series.index)

    history = series.iloc[t-lags:t]
    y_true  = series.iloc[t:t+horizon]
    if len(history) < lags or len(y_true) < horizon:
        return

    idx_t = series.index[t]
    cal = np.array([idx_t.hour, idx_t.dayofweek, int(idx_t.dayofweek >= 5), idx_t.month])
    met = meteo.loc[idx_t, meteo_features].values

    X = np.concatenate([history.values, cal, met]).reshape(1, -1)
    models = models_dict[meter]
    y_pred = np.array([models[h].predict(X)[0] for h in range(horizon)])

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=history.index, y=history.values, mode="lines+markers",
                             name="History (model input)", line=dict(color="black")))
    fig.add_trace(go.Scatter(x=y_true.index, y=y_pred, mode="lines+markers",
                             name="LGBM Multi Prediction", line=dict(color="red", dash="dash")))
    fig.add_trace(go.Scatter(x=y_true.index, y=y_true.values, mode="lines+markers",
                             name="True", line=dict(color="blue")))

    fig.update_layout(
        title=f"LGBM Multivariate Forecast — Meter {meter}",
        template="plotly_white",
        xaxis_title="Time",
        yaxis_title="Energy",
        height=500,
        autosize = True
    )
    fig.show(config={"responsive": True})

In [ ]:
meter = "PCC"
lags = 24
horizon = 24

series_len = len(train[meter]) + len(test[meter])

t_slider = widgets.IntSlider(
    value=len(train[meter]),
    min=lags,
    max=series_len - horizon,
    step=1,
    description="t:",
    continuous_update=False
)

play = widgets.Play(
    value=t_slider.value,
    min=t_slider.min,
    max=t_slider.max,
    step=t_slider.step,
    interval=300
)

widgets.jslink((play, "value"), (t_slider, "value"))
ui = widgets.HBox([play, t_slider])

out = widgets.interactive_output(
    plot_lgbm_multi_window,
    {
        "meter": widgets.fixed(meter),
        "t": t_slider,
        "models_dict": widgets.fixed(lgbm_multivariate_models),
        "meteo_df": widgets.fixed(df_meteo),
        "meteo_features": widgets.fixed(meteo_features),
        "lags": widgets.fixed(lags),
        "horizon": widgets.fixed(horizon),
    }
)

display(ui, out)


#### 3.2.3. Comparison

In [ ]:
def plot_lgbm_comparison_window(
    meter, t,
    models_uni, models_multi,
    meteo_df, meteo_features,
    lags=24, horizon=24
):
    series = pd.concat([train[meter], test[meter]])
    meteo  = meteo_df.reindex(series.index)

    history = series.iloc[t-lags:t]
    y_true  = series.iloc[t:t+horizon]

    if len(history) < lags or len(y_true) < horizon:
        return

    # ---------- UNIVARIATE ----------
    X_uni = history.values.reshape(1, -1)
    y_pred_uni = np.array([
        models_uni[meter][h].predict(X_uni)[0]
        for h in range(horizon)
    ])

    # ---------- MULTIVARIATE ----------
    idx_t = series.index[t]
    cal = np.array([idx_t.hour, idx_t.dayofweek, int(idx_t.dayofweek >= 5), idx_t.month])
    met = meteo.loc[idx_t, meteo_features].values
    X_multi = np.concatenate([history.values, cal, met]).reshape(1, -1)

    y_pred_multi = np.array([
        models_multi[meter][h].predict(X_multi)[0]
        for h in range(horizon)
    ])

    # ---------- PLOT ----------
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=history.index, y=history.values,
        mode="lines+markers",
        name="History (model input)",
        line=dict(color="black")
    ))

    fig.add_trace(go.Scatter(
        x=y_true.index, y=y_pred_uni,
        mode="lines+markers",
        name="LGBM Univariate",
        line=dict(color="red", dash="dash")
    ))

    fig.add_trace(go.Scatter(
        x=y_true.index, y=y_pred_multi,
        mode="lines+markers",
        name="LGBM Multivariate",
        line=dict(color="green", dash="dash")
    ))

    fig.add_trace(go.Scatter(
        x=y_true.index, y=y_true.values,
        mode="lines+markers",
        name="True",
        line=dict(color="blue")
    ))

    fig.update_layout(
        title=f"LGBM Comparison (Uni vs Multi) — Meter {meter}",
        template="plotly_white",
        xaxis_title="Time",
        yaxis_title="Energy",
        height=520,
        autosize = True
    )
    fig.show(config={"responsive": True})

In [ ]:
meter = "PCC"
lags = 24
horizon = 24

series_len = len(train[meter]) + len(test[meter])

t_slider = widgets.IntSlider(
    value=len(train[meter]),
    min=lags,
    max=series_len - horizon,
    step=1,
    description="t:",
    continuous_update=False
)

play = widgets.Play(
    value=t_slider.value,
    min=t_slider.min,
    max=t_slider.max,
    step=t_slider.step,
    interval=300
)

widgets.jslink((play, "value"), (t_slider, "value"))

ui = widgets.HBox([play, t_slider])

out = widgets.interactive_output(
    plot_lgbm_comparison_window,
    {
        "meter": widgets.fixed(meter),
        "t": t_slider,
        "models_uni": widgets.fixed(all_lgbm_models),
        "models_multi": widgets.fixed(lgbm_multivariate_models),
        "meteo_df": widgets.fixed(df_meteo),
        "meteo_features": widgets.fixed(meteo_features),
        "lags": widgets.fixed(lags),
        "horizon": widgets.fixed(horizon),
    }
)

display(ui, out)


In [ ]:
order = [i for i in range(20)]
if "battery" in lgbm_errors: order.append("battery")
if "PCC" in lgbm_errors: order.append("PCC")

extras = [k for k in lgbm_errors.keys() if k not in order]
order += [k for k in extras]

df_mae = pd.DataFrame({
    "LGBM Univariate": [lgbm_errors.get(m, np.nan) for m in order],
    "LGBM Multivariate": [lgbm_multivariate_errors.get(m, np.nan) for m in order],
}, index=order)

fig = go.Figure()

fig.add_bar(
    x=df_mae.index.astype(str),
    y=df_mae["LGBM Univariate"],
    name="LGBM Univariate",
    marker_color="red"
)

fig.add_bar(
    x=df_mae.index.astype(str),
    y=df_mae["LGBM Multivariate"],
    name="LGBM Multivariate",
    marker_color="green"
)

mu = df_mae["LGBM Univariate"].mean(skipna=True)
mm = df_mae["LGBM Multivariate"].mean(skipna=True)

fig.update_layout(
    title=f"MAE comparison per meter (avg uni={mu:.3f}, avg multi={mm:.3f})",
    xaxis_title="Meter",
    yaxis_title="MAE",
    barmode="group",
    template="plotly_white",
    height=520
)

fig.show()


### 3.3. Chronos Model

#### 3.3.1. Univariate

In [ ]:
from chronos import Chronos2Pipeline

pipeline = Chronos2Pipeline.from_pretrained(
    "amazon/chronos-2",
    device_map="cpu",
)

In [ ]:
def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def chronos_uni_mae(series_train, series_test, horizon=24, context=168, n_eval=200):
    s = np.concatenate([series_train.values, series_test.values]).astype("float32")
    n_total = len(s)

    errs = []
    max_i = n_total - (context + horizon)
    idxs = np.linspace(0, max_i, num=min(n_eval, max_i + 1), dtype=int)

    for i in idxs:
        ctx = s[i:i+context]                 
        y_true = s[i+context:i+context+horizon]

        x = ctx[None, None, :]               

        out = pipeline.predict(
            x,
            prediction_length=horizon,
            context_length=context, 
        )
        pred = out[0]  
        try:
            pred = pred.detach().cpu().numpy()
        except Exception:
            pass
        y_pred = np.asarray(pred).reshape(-1)[:horizon]
        errs.append(np.mean(np.abs(y_true - y_pred)))

    return float(np.mean(errs))


In [ ]:
chronos_uni_errors = {}

for meter in meters:
    err = chronos_uni_mae(train[meter], test[meter], horizon=24, context=168, n_eval=200)
    chronos_uni_errors[meter] = err
    print(f"Chronos2 (uni) MAE meter {meter}: {err:.3f}")

print("\nAverage Chronos2 (uni) MAE across all meters:", round(np.mean(list(chronos_uni_errors.values())), 3))

In [ ]:
def plot_chronos_uni_window(meter, t, horizon=24, context=168):
    series = pd.concat([train[meter], test[meter]]).dropna()

    history = series.iloc[t-context:t]
    y_true  = series.iloc[t:t+horizon]
    if len(history) < context or len(y_true) < horizon:
        return

    ctx = history.values.astype("float32")[None, None, :]  # (1,1,context)

    out = pipeline.predict(ctx, prediction_length=horizon, context_length=context)

    pred = out[0]
    try:
        pred = pred.detach().cpu().numpy()
    except Exception:
        pass
    y_pred = np.asarray(pred).reshape(-1)[:horizon]

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=history.index, y=history.values, mode="lines+markers",
                             name="History (model input)", line=dict(color="black")))
    fig.add_trace(go.Scatter(x=y_true.index, y=y_pred, mode="lines+markers",
                             name="Chronos2 Prediction", line=dict(color="red", dash="dash")))
    fig.add_trace(go.Scatter(x=y_true.index, y=y_true.values, mode="lines+markers",
                             name="True", line=dict(color="blue")))

    fig.update_layout(
        title=f"Chronos2 Univariate Forecast — Meter {meter}",
        template="plotly_white",
        xaxis_title="Time",
        yaxis_title="Energy",
        height=500,
        autosize=True
    )
    fig.show(config={"responsive": True})


In [ ]:
meter = "PCC"
horizon = 24
context = 168

series_len = len(train[meter]) + len(test[meter])

t_slider = widgets.IntSlider(
    value=len(train[meter]),
    min=context,
    max=series_len - horizon,
    step=1,
    description="t:",
    continuous_update=False
)

play = widgets.Play(
    value=t_slider.value,
    min=t_slider.min,
    max=t_slider.max,
    step=t_slider.step,
    interval=300
)

widgets.jslink((play, "value"), (t_slider, "value"))
ui = widgets.HBox([play, t_slider])

out = widgets.interactive_output(
    plot_chronos_uni_window,
    {"meter": widgets.fixed(meter), "t": t_slider,
     "horizon": widgets.fixed(horizon), "context": widgets.fixed(context)}
)

display(ui, out)


#### 3.3.1. Multivariate

In [ ]:
def chronos_multi_mae(train_df, test_df, meters, target_meter, horizon=24, context=168, n_eval=200):
    full = pd.concat([train_df[meters], test_df[meters]], axis=0).astype("float32")
    j = meters.index(target_meter)

    errs = []
    max_i = len(full) - (context + horizon)
    idxs = np.linspace(0, max_i, num=min(n_eval, max_i + 1), dtype=int)

    for i in idxs:
        hist = full.iloc[i:i+context]
        fut  = full.iloc[i+context:i+context+horizon]

        if hist.isna().any().any() or fut.isna().any().any():
            continue

        x = hist.to_numpy(dtype="float32").T[None, :, :]  # (1, n_meters, context)

        out = pipeline.predict(
            x,
            prediction_length=horizon,
            context_length=context,
        )

        pred = out[0]
        try:
            pred = pred.detach().cpu().numpy()
        except Exception:
            pass

        y_pred = np.asarray(pred)[0, j, :horizon]
        y_true = fut[target_meter].to_numpy(dtype="float32")

        errs.append(mae(y_true, y_pred))

    return float(np.mean(errs)) if len(errs) > 0 else np.nan

In [ ]:
chronos_multi_errors = {}

for meter in meters:
    err = chronos_multi_mae(train, test, meters, target_meter=meter, horizon=24, context=168, n_eval=200)
    chronos_multi_errors[meter] = err
    print(f"Chronos2 (multi) MAE meter {meter}: {err:.3f}")

print("\nAverage Chronos2 (multi) MAE across all meters:", round(np.nanmean(list(chronos_multi_errors.values())), 3))

In [ ]:
def plot_chronos_multi_window(meter, t, horizon=24, context=168):
    full = pd.concat([train[meters], test[meters]], axis=0).astype("float32")

    history = full.iloc[t-context:t]           # (context, n_meters)
    y_true  = full.iloc[t:t+horizon]           # (horizon, n_meters)

    if len(history) < context or len(y_true) < horizon:
        return
    if history.isna().any().any() or y_true.isna().any().any():
        return

    x = history.to_numpy(dtype="float32").T[None, :, :]   # (1, n_meters, context)

    out = pipeline.predict(x, prediction_length=horizon, context_length=context)

    pred = out[0]
    try:
        pred = pred.detach().cpu().numpy()
    except Exception:
        pass
    pred = np.asarray(pred)

    # (1, n_meters, horizon) -> seleziona il meter richiesto
    j = meters.index(meter)
    y_pred = pred[0, j, :horizon]

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=history.index, y=history[meter].values, mode="lines+markers",
        name="History (model input)", line=dict(color="black")
    ))
    fig.add_trace(go.Scatter(
        x=y_true.index, y=y_pred, mode="lines+markers",
        name="Chronos2 Prediction (multivariate)", line=dict(color="red", dash="dash")
    ))
    fig.add_trace(go.Scatter(
        x=y_true.index, y=y_true[meter].values, mode="lines+markers",
        name="True", line=dict(color="blue")
    ))

    fig.update_layout(
        title=f"Chronos2 Multivariate Forecast — Meter {meter}",
        template="plotly_white",
        xaxis_title="Time",
        yaxis_title="Energy",
        height=500,
        autosize=True
    )
    fig.show(config={"responsive": True})


In [ ]:
meter = "PCC"
horizon = 24
context = 168

series_len = len(train[meters]) + len(test[meters])

t_slider = widgets.IntSlider(
    value=len(train[meters]),
    min=context,
    max=series_len - horizon,
    step=1,
    description="t:",
    continuous_update=False
)

play = widgets.Play(
    value=t_slider.value,
    min=t_slider.min,
    max=t_slider.max,
    step=t_slider.step,
    interval=300
)

widgets.jslink((play, "value"), (t_slider, "value"))
ui = widgets.HBox([play, t_slider])

out = widgets.interactive_output(
    plot_chronos_multi_window,
    {"meter": widgets.fixed(meter), "t": t_slider,
     "horizon": widgets.fixed(horizon), "context": widgets.fixed(context)}
)

display(ui, out)


#### 3.3.1. Comparison

## MODEL SELCTION

## CROSS VALIDATIO AND TUNING IN CASE

## FINAL EVALUATION OF THE RESULTS 